Twitter Soccer Analysis
============
*Insert description here*

Data Overview
--------------
    Technical Data:
    Instance   : Amazon EC2 (Ubuntu, m4.large)  
    Database   : Amazon RDS (MySQL, t2.large)
    Python     : 2.7 (Not working in python 3)

    About the Data:
    Souce      : Twitter Streaming API
    Type       : JSON
    Num. Rows  : 127,607  
    Size       : approx. 1.6 GigaByte

Preparation
-----------
In order to get data from database, we need to install MySQLdb first. MySQLdb is one of python library that provide driver or direct connection to MySQL database. As a result, all what we need is import MySQLdb into our machine.

**Super User**  
We need to have access to sudo command, so we can run installation command smoothly.

In [ ]:
!sudo su

**Installing Anaconda**  
We need to install Anaconda just in case if compatibility issue happens

In [ ]:
!wget https://3230d63b5fc54e62148e-c95ac804525aac4b6dba79b00b39d1d3.ssl.cf1.rackcdn.com/Anaconda2-2.4.0-Linux-x86_64.sh
!bash Anaconda2-2.4.0-Linux-x86_64.sh

The process above is to download data from Anaconda website and to install Anaconda after downloading the data.

**Installing MySQLdb**  
Before going deeper to access database, we need to install MySQLdb first.

In [ ]:
!pip install pip --upgrade
!apt-get build-dep python-mysqldb
!pip install MySQL-python

**Other Works**  
We also did some of dirty works called **`Data Preparation`** such as:
    * Getting and storing Twitter streaming data
    * Extracting and counting, words and location based on timezone
    * Cleaning words and location data
If you are interested in "How to we did that?" you may also uncomment the extra code below.

> *please be noted that the code below assuming using the same Amazone RDS. The code may fail if Amazon RDS is not available*

**Getting and storing Twitter streaming data**  
Before getting twitter streaming data, we need to install TwitterAPI library. Please be noted that our Amazon RDS server is still active by the time we wrote this note. If you find the server is not active, please do not run this code.

In [ ]:
!pip install TwitterAPI

After completing installation, uncomment this code to fetch real time Twitter Data

In [ ]:
'''
from TwitterAPI import TwitterAPI
import MySQLdb
import json

def insert_into_db(raw):
    db = MySQLdb.connect(host="mydb.c1xbtz6tteqx.us-east-1.rds.amazonaws.com", user="frans", passwd="smu.frans", db="TwitterData")
    cur = db.cursor()
    raw = raw.encode('string_escape')
    cur.execute("INSERT INTO Raw(Data) VALUES(%s)",[raw])
    db.commit()
    
consumer_key = "iMzT16JgLSyy6gaZNGjzdyoC4"
consumer_secret = "bE22hKUHSEK7aiUkvemFxc6Mg5qk8pWCAT06pq7OqPSQZTgRwI"
access_token_key = "161989718-UU1ZPTlcUMreZrmA6evrIvpEb7tgT9AfxVrcWoMF"
access_token_secret = "IasUHu42sqcbm3sjAkvTj45rWHxXZ7jV3VdVWSSCLvvqL"

api = TwitterAPI(consumer_key, consumer_secret, access_token_key, access_token_secret)

r = api.request('statuses/filter', {'track':'epl,bpl'})
for item in r:
    raw = json.dumps(item);
    insert_into_db(raw)
    print(raw)
'''

# Extracting Data
We collected the raw data from Twitter Streaming Data. Next step is to extract data. We have to extract data from raw data. There are 3 different type of extarction that we should do.  
    1. Words Extraction
    2. Locations Extraction
    3. Words and Loctions Extractions

**Words Extraction**  
Words extraction is extracting data based on every words. We count every word occurence in each tweet. For example `Liver Pool [10]`. "Liver Pool" had occured for 10 times.

In order to achive this approach, we need to do some coding. Below is the coding.

*NOTE: Please uncomment to run the code. Running the code will take so long time*

In [ ]:
'''

'''

**Locations Extraction**  
Location is the same as Words Extraction. Both of extraction method also counting the number of occurence. The main difference in this method is Location Extraction use *Timezone* data.

Code is provided below. Please uncomment to run the code. *Running the code will take so long time*

In [ ]:
'''
'''

**Words and Locations Extraction**  
This method is combine between Words Extraction and Locations Extraction methods. This method searching in which location did the word said.

Code is provided below. Please uncomment to run the code. *Running the code will take so long time*

In [ ]:
'''
import json
import MySQLdb
import sys
#Connecto to Databsae
db = MySQLdb.connect(host="mydb.c1xbtz6tteqx.us-east-1.rds.amazonaws.com", user="frans", passwd="smu.frans", db="TwitterData")
cur = db.cursor()

#Execute SQL Command to get all the data
cur.execute("SELECT * FROM Raw")
for a in cur.fetchall():
    try:
        #extract the data from database
        mydata = a[1].replace("\\\\","\\")
        myjson = json.loads(mydata)
        
        #extract text and split the text by space (" ")
        text = myjson["text"]
        text = text.split(" ")
        
        #extract location
        location = myjson["user"]["time_zone"]
        
        #store the word with the location
        for word in text:
            #find the count of word and location in database
            #if not found, create a new one
            #if found, just increment the count
            cur2 = db.cursor()
            cur2.execute("SELECT COUNT(*) FROM WordsLocation WHERE Words=%s AND Location=%s",[word,location])
            count = cur2.fetchone()
            count = count[0]
            if(count==0):
                #insert into databaes
                cur2.execute("INSERT INTO WordsLocation(Words,Location,Count) VALUES(%s,%s,%s)",[word,location,1])
            else:
                cur2.execute("SELECT Count FROM WordsLocation WHERE Words=%s AND Location=%s",[word,location])
                count = cur2.fetchone()
                count = count[0]
                count = count + 1
                cur2.execute("UPDATE WordsLocation SET Count=%s WHERE Words=%s AND Location=%s",[count,word,location])
            db.commit()
    except:
        print("Error: {}".format(sys.exc_info()[0]))
'''

<b style="color:red">We did comment the process because the process can take more than 1 day. However, we provide you the intersting part! Just continue to read the docummentation below</b>

# Installing plotly and dataseet
we need to install plotly in order to make the plot works. To do that, we have to install it using this command
```python
sudo pip install plotly
sudo pip install dataset
```

# Location Analysis
The purpose of this analysis is to know which country has the most tweet. Run this code to know better. You need internet connection and `plotly` in order to run this code.

In [4]:
import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls
import plotly.graph_objs as go
import MySQLdb

tls.set_credentials_file(username='DemoAccount', api_key='lr1c37zw81')

# get data from database
db = MySQLdb.connect(host="cloudandbigdata.cwkktstj3vyr.ap-southeast-1.rds.amazonaws.com", user="frans", passwd="smu.frans", db="TwitterData")
cur = db.cursor()

cur.execute("SELECT * FROM Location ORDER BY Count DESC LIMIT 20")
data = {}
data["index"] = []
data["count"] = []
mydata = {}
mydata["index"] =[]
mydata["count"] = []
for a in cur.fetchall():
    try:
        data["index"].append(a[0])
        data["count"].append(a[1])
    except:
        pass

data = [
    go.Bar(
        x=data["index"],
        y=data["count"]
    )
]
py.iplot(data, filename='basic-bar')

# Words Analysis
We did find which word are most used by people in Twitter. Run this code to find out the result